In [1]:
import math
import numpy as np
import csv
import json
import pandas as pd
import copy
from collections import OrderedDict

In [70]:
class System():
    #どうするかな？
    def __init__(self, sat):
        self.sat = sat
        self.candidates = {}
        self.total_candidates = {COM_ID : {"COM":[], "TEL":[]} for COM_ID in self.sat.COM.keys()}
        self.effectness = {COM_ID : {} for COM_ID in self.sat.COM.keys()}
        self.negative_effect = {COM_ID : {} for COM_ID in self.sat.COM.keys()}
        #初期値どうするべきか
        self.human_select = 0
        self.selectedCOM = []
        self.remainingCOM = [ID for ID in self.sat.COM.keys()]
    
    def init_element(self, df):
        self.elements = []
        #とりあえずlistに収める形だけ実装．細かい体裁は下位で実装
        for row in df.itertuples():
            self.elements.append(row)
            
    def verify(self):
        #検証していくメインのメソッド
        print(1)

        #テレメトリの確認で済むものも提示して，結果がどうなのかを受け取るようにしたい

        #選択肢を表示させる．この時に第一段階（波及効果を考えないもの）全てを与えるのか，影響が小さいものから表示するのか？
        #提示するものは検証できるパスがあるものだけ．当たり前やが
        #後者が望ましいができるのか？まずは単純に通るパスの長さ．あとコマンドの種別（Action, Set, Get, Mem）

        
        #選択を受けて更新する．この時にOKだったのかどうかも必要そう
        #OKであれば検証したとして除けばいいが，正常な挙動でなかったときはどうする？
        #そこを深堀するのか，第一段階で残っている検証可能性を潰していくのか？それも状況によるのかもしれない．．．
        #それらの情報によって次の選択肢を表示，

        #選択肢がなくなる or targetがなくなれば終了して，最後に残ったものを提示
        

    def verify_plan(self):
    #まずはテレメトリのみでの確認ができるか判定．つまりコマンドによる変化なしで状態変化する状態量の確認
    #この時にtargetTELで与えられたものは除く
    #テレメトリに関して，下ろしているものなのかそうでないのかを考えていない
        candidate_TEL_ID = []
        for TEL_ID in self.sat.TEL.keys():
            if (not self.sat.targetCOMpath and not self.sat.targetTELpath):
                return 0
            elif TEL_ID in self.sat.targetTEL_ID:
                continue
            elif self.sat.TEL[TEL_ID].triger == 'Time':
                #TELによる検証
                self.verify_by_TEL(TEL_ID)
                
            #コマンドがトリガーのものでもinitial_COMにあるものなら見れるが，後に見る
            elif self.sat.TEL[TEL_ID].triger == 'Command':
                continue
        print("\n Check telemetries which influenced by initial Command state\n")
        for iniCOM_ID in self.sat.initial_COM:
            #この時の更新はiniCOMなので入力関係ない．結果の確認だけ必要
            #この前になにを確認するのかという表示が必要
            self.verify_by_COM(iniCOM_ID)
            #このコマンドが検証できるポート数を数える．
            
        #コマンドによる検証に入る．
        while(1):
            if(self.verify_by_COM()):
                print("finish")
                break
            print("selected Command:",self.selectedCOM,"remaining Command:",self.remainingCOM)
    
    def find_total_link(self, COM_ID):
        #初期化
        self.total_candidates[COM_ID]["COM"] = []
        self.total_candidates[COM_ID]["TEL"] = []
        for key in self.candidates.keys():
            #telに関するものは飛ばす
            if (len(key)<2):
                continue
            elif key[0]!=COM_ID:
                continue
            else:
                self.total_candidates[COM_ID]["COM"] = list(set(self.total_candidates[COM_ID]["COM"] + \
                self.candidates[key]["COM"]))
                self.total_candidates[COM_ID]["TEL"] = list(set(self.total_candidates[COM_ID]["TEL"] + \
                self.candidates[key]["TEL"]))
                self.effectness[COM_ID]["veri_link_num"] = len(self.total_candidates[COM_ID]["COM"]) + \
                len(self.total_candidates[COM_ID]["TEL"])
                #print(self.candidates[key]["veri_link_num"])
                
    def verify_by_TEL(self, TEL_ID):
        self.candidates.update(self.sat.search_TEL(TEL_ID))
        #verifyできるものがない場合ははじく
        if not self.candidates[(TEL_ID,)]["TEL"]:
            return 0
        print("TELtarget:",self.sat.targetTELpath)
        print("Telemetry",TEL_ID,"(",self.sat.TEL[TEL_ID].name, ") can verify following links\n", self.candidates[(TEL_ID,)]["TEL"])
        #まずここで確認してもらうか
        self.human_select = TEL_ID
        self.receive_results(TEL_ID)
        self.update_target_path("TEL")
        
    #オーバーロードする
    def verify_by_COM(self, COM_ID=0):
        if not self.remainingCOM:
            return 1
        
        #現状の提示
        print("COMtarget:",self.sat.targetCOMpath,"TELtarget:",self.sat.targetTELpath)
        if not COM_ID:
            #初期化
            for COMp in self.sat.COMpinks.values():
                COMp.verifyCOMnum = 0
            for TELp in self.sat.TELlinks.values():
                TELp.verifyCOMnum = 0
            #探索開始
            for COM_ID in self.remainingCOM:
                if COM_ID in (self.sat.targetCOM_ID or self.selectedCOM): 
                    continue
                #残りがなくなるか，対象がなくなれば終了
                elif (not self.sat.targetCOMpath and not self.sat.targetTELpath):
                    print("nothing target")
                    return 1
                else:
                    self.candidates.update(self.sat.find_check_COM(COM_ID))
                    self.find_total_link(COM_ID)
                    #現状でのコマンドによって検証できる箇所の洗い出しが終了したので，
                    #あるポートを検証できるコマンドの数を調べる．
                    self.count_COM_num_for_link(COM_ID)
                    self.propagate_COM_effect(COM_ID)
                    #電力がマイナスになるものは禁止
                    if not self.calculate_point(COM_ID):
                        print("NG")
                        continue
                        #選択肢として表示しないような処理必要
                    #ここでポイント表示したい
                    self.show_point(COM_ID)
                    #残りはあるけど，検証に使えないときは？
                    #print(COM_ID,":backup num", self.sat.COM[COM_ID].candidateTELnum)
            flag = 0
            for remainCOM in self.remainingCOM:
                #flagが立ってたら，次に行ける
                if self.sat.COM[remainCOM].verify_flag:
                    flag = 1
                    break
                else:
                    continue
            #全部通過したら終了
            if not flag:
                print("nothing can verify")
                return 1
            
            #ここが実際の検証を行うところ
            self.receive_selection()
        #iniCOM用
        else:
            if COM_ID in self.selectedCOM: 
                return 1
            #残りがなくなるか，対象がなくなれば終了
            elif (not self.sat.targetCOMpath and not self.sat.targetTELpath):
                return 1
            else:
                self.candidates.update(self.sat.find_check_COM(COM_ID))
                self.find_total_link(COM_ID)
                self.count_COM_num_for_link(COM_ID)
            self.receive_selection(COM_ID)
    
        self.update_target_path("COM")
    
    def count_COM_num_for_link(self,COM_ID):
        #COM
        for COMp in self.sat.COMpinks.values():
            if COMp.ID in self.total_candidates[COM_ID]["COM"]:
                COMp.verifyCOMnum = COMp.verifyCOMnum + 1
        #TEL
        for TELp in self.sat.TELlinks.values():
            if TELp.ID in self.total_candidates[COM_ID]["TEL"]:
                TELp.verifyCOMnum = TELp.verifyCOMnum + 1
            
    def calculate_point(self,COM_ID):
        #レア度を計算
        allCOMnum = len(self.sat.COM)
        self.effectness[COM_ID]["COMrareness"] = 0
        for COMp_ID in self.total_candidates[COM_ID]["COM"]:
            probabiity = float(self.sat.COMpinks[COMp_ID].verifyCOMnum)/allCOMnum
            self.effectness[COM_ID]["COMrareness"] = self.effectness[COM_ID]["COMrareness"] + \
            -probabiity*math.log(probabiity,allCOMnum)
            
        #allTELnum = len(self.sat.TEL)
        for TELp_ID in self.total_candidates[COM_ID]["TEL"]:
            probabiity = float(self.sat.TELlinks[TELp_ID].verifyCOMnum)/allCOMnum
            self.effectness[COM_ID]["COMrareness"] = self.effectness[COM_ID]["COMrareness"] + \
            -probabiity*math.log(probabiity,allCOMnum)
        #引き出しの多さ
        self.effectness[COM_ID]["back_up_TELnum"] = self.sat.COM[COM_ID].candidateTELnum
        #以下でネガティブポイントを計算
        #電力へのインパクト
        #RemainingRatio = self.sat.PreviousRemainPower/self.sat.TotalPower
        #ConsumeRatio = (self.sat.RemainingPower - self.sat.PreviousRemainPower)/self.sat.PreviousRemainPower
        
        if self.sat.RemainingPower < 0:
            return 0
        #ここは現在の状態に応じて計算を変えないといけない．というか，状態を変化させないコマンドを飛ばすようにすればいい．
        self.negative_effect[COM_ID]["Remaining Power"] = self.sat.RemainingPower + self.sat.COM_consume_power
        self.negative_effect[COM_ID]["Power consume by this COM"] = self.sat.COM_consume_power
        return 1
        
    def show_candidates(self):
        for key in self.candidates.keys():
            #telに関するものは飛ばす
            if (len(key)<2):
                continue
            elif key[0]!=self.human_select:
                continue
            elif not self.candidates[key]["COM"] and not self.candidates[key]["COM"]:
                continue
            print("Command",key[0],"(",self.sat.COM[key[0]].name, ") & Telemetry",\
                  key[1],"(", self.sat.TEL[key[1]].name, ") can verify following links\n",\
              "COMpink:",self.candidates[key]["COM"], "TELlink", self.candidates[key]["TEL"])
        #print("check those corresponding Telemetry")

    #デフォルト値使ってオーバーロードする
    def receive_selection(self, COM_ID=0):
        if not COM_ID:
            while(1):
                self.human_select = int(input("Please select Command above(input ID) >>"))
                #ここに想定してない入力をはじく処理必要
                if (self.human_select not in self.remainingCOM):
                    continue
                else:
                    self.show_candidates()
                    break
        else:
            self.human_select = COM_ID
            self.show_candidates()
        self.selectedCOM.append(self.human_select)
        self.remainingCOM = list(set(self.remainingCOM) - set(self.selectedCOM))
    
    def receive_results(self,TEL_ID):
        while(1):
            #初めに入力を受け付ける
            print("\nPlease check", self.sat.TEL[TEL_ID].name)
            self.result = input("Input result(OK or NG)>>")
            if (self.result!="OK" and self.result!="NG"):
                self.result = input("Please input OK or NG >>")
                if (self.result!="OK" and self.result!="NG"):
                    continue
                else:
                    break
            else:
                break
        #チェックしたものを再度確認しないようにフラグ必要←これいる？
        #self.sat.TEL[TEL_ID].checked_flag = 1
    
    def update_target_path(self, TELorCOM):
        if(TELorCOM=="COM"):
            COM_ID = self.human_select
            #ここで確認した組み合わせは不要だからcandidatesから消す？
            for TEL_ID in self.sat.COM[COM_ID].impact_telID:
                select_key = (COM_ID,TEL_ID)
                #不具合検知の組み合わせ 
                if TEL_ID in self.sat.targetTEL_ID and COM_ID in self.sat.targetCOM_ID:
                    continue
                #verify link なし
                elif (not self.candidates[select_key]["COM"] and not self.candidates[select_key]["TEL"]):
                    continue
                #確認してもらう
                else:
                    self.receive_results(TEL_ID)
                if(self.result=="NG"):
                    continue
                else:
                    #print(self.candidates[select_key])
                    for COMpinkID in self.candidates[select_key][TELorCOM]:
                        self.sat.COMpinks[COMpinkID].valid = 1
                        #print("veified COMpinkID:",COMpinkID)
                    for TELlinkID in self.candidates[select_key]["TEL"]:
                        self.sat.TELlinks[TELlinkID].valid = 1
                        #print("verified TELlinkID:",TELlinkID)
                print("COMpink:",self.candidates[select_key][TELorCOM], "& TELlink:",\
                      self.candidates[select_key]["TEL"], "were verified")
                #targetの更新
                self.sat.targetCOMpath = self.sat.check_links(self.sat.targetCOMpath, self.sat.COMpinks)
                self.sat.targetTELpath = self.sat.check_links(self.sat.targetTELpath, self.sat.TELlinks)
                #candidate update必要
                self.candidates.update(self.sat.find_check_COM(COM_ID))
                #ここでも集計し直す
                self.find_total_link(COM_ID)
                self.count_COM_num_for_link(COM_ID)
                self.propagate_COM_effect(COM_ID)
                self.calculate_point(COM_ID)
                #ここでポイント表示したい
                self.show_point(COM_ID)
            #確認フラグを戻す
            for TEL_ID in self.sat.COM[COM_ID].impact_telID:
                self.sat.TEL[TEL_ID].checked_flag = 0
            return 1
        else:
            select_key = (self.human_select,)
            if(self.result=="NG"):
                return 0
            else:
                #各ポートのvalidを更新
                for p_ID in self.candidates[select_key][TELorCOM]:
                    self.sat.TELlinks[p_ID].valid = 1
                    #print("verified TELlinkID:",p_ID)
                self.sat.targetTELpath = self.sat.check_links(self.sat.targetTELpath, self.sat.TELlinks)
                return 1
            
    #これを使って表示する前に効果と波及効果の大きさをもとにソートして，一部だけの表示にする．
    def show_point(self, COM_ID):
        #既に打ったものは表示しない
        if COM_ID in self.selectedCOM:
            return 0
        elif not self.total_candidates[COM_ID]["COM"] and not self.total_candidates[COM_ID]["TEL"]:
            return 0
        #効果を表示
        print("COM", COM_ID, self.sat.COM[COM_ID].name, self.effectness[COM_ID],self.negative_effect[COM_ID])
        
        #そのコマンドを打つとどうなるかを更新する
    #これはあくまでも可能性の更新．本当にそのような遷移をしたかは，テレメトリの結果に依存する．←これはどうやって実装する？
    def propagate_COM_effect(self,COM_ID):
        previous_compo_state = {}
        if self.sat.COM[COM_ID].type == "ACTION":
            for target in self.sat.COM[COM_ID].target:
                compo = self.sat.compos[target["Component"]]
                previous_compo_state[compo.name] = copy.deepcopy(compo.Active)
                #電源操作系のコマンドはFunctionが空
                if not target["Function"]:
                    if self.sat.COM[COM_ID].Active and not compo.Active:
                        compo.Active = True
                        self.sat.COM_consume_power = compo.PowerConsumption
                    elif not self.sat.COM[COM_ID].Active and compo.Active:
                        compo.Active = False
                        self.sat.COM_consume_power = - compo.PowerConsumption
                    else:
                        self.sat.COM_consume_power = 0
                #コマンドがFunctionを持つ場合
                else:
                    for func in target["Function"]:
                        #コンポーネントはFunctionの名前をキーとして持つ
                        if self.sat.COM[COM_ID].Active and not compo.Function[func]["Active"]:
                            compo.Function[func]["Active"] = True
                            self.sat.COM_consume_power = compo.Function[func]["PowerConsumption"]
                        elif not self.sat.COM[COM_ID].Active and compo.Function[func]["Active"]:
                            compo.Function[func]["Active"] = False
                            self.sat.COM_consume_power = - compo.Function[func]["PowerConsumption"]
                        else:
                            self.sat.COM_consume_power = 0
            self.sat.update_Power_state()
            #もどす
            for name in previous_compo_state.keys():
                self.sat.compos[name].Active = previous_compo_state[name]
        #今は未実装なだけ，GETとかを実装する
        else:
            return 0
    

class Satellite(System):
    #各要素は辞書型で格納，アクセスするための参照をkeyにしておく,valueはオブジェクト
    def __init__(self, compos_df, links_df, COM_df, TEL_df, COM_type_dict):
        super().init_element(compos_df)
        self.compos = {x.name : x for x in [Component(x) for x in self.elements]}
        super().init_element(links_df)
        self.all_links = {x.ID : x for x in [Link(x) for x in self.elements]}
        self.COMpinks = {}
        self.TELlinks = {}
        self.classify_links()
        #print(self.TELlinks)
        #COMとTELだけpathの短い順に保持したい
        super().init_element(COM_df)
        COM_dict = OrderedDict({x.ID : x for x in [Command(x) for x in self.elements]})
        self.COM = OrderedDict(sorted(COM_dict.items(), key=lambda x:len(x[1].path)))
        #コマンドの種別の初期化
        for COM in self.COM.values():
            self.COM[COM.ID].init_COM_type(COM_type_dict[COM.name])
        super().init_element(TEL_df)
        TEL_dict = OrderedDict({x.ID : x for x in [Telemetry(x) for x in self.elements]})
        self.TEL = OrderedDict(sorted(TEL_dict.items(), key=lambda x:len(x[1].path)))
        #PCU or BATが持つ電力容量を衛星が管理
        self.RemainingPower = 0
        self.ConsumingPower = 0
        self.targetCOMpath = []
        self.targetTELpath = []
        
        
    #初期状態が必要．何のコマンドを受け取っていたのか？(どの機器がONになっていたのか？)ここの洗練化
    def init_state(self, ini_COMpist, all_compo_state_dict):
        self.initial_COM = ini_COMpist
        #コンポのstate追加
        for compo in self.compos.values():
            compo.update_state(all_compo_state_dict[compo.name])
        #衛星全体の電力状態を初期化
        self.TotalPower = self.compos["BAT"].state["PowerAmount"]["value"]
        self.update_Power_state()
        #最初だけ初期化後にコピー
        self.PreviousRemainPower = copy.deepcopy(self.RemainingPower)
        
    def update_Power_state(self):
        #全部に関して回すため，一回リセット
        self.RemainingPower = self.TotalPower #全体の容量を足す．ほんまは単位が別[mAh]なので足し合わせることはできないが，簡単のため今はこれでいい
        self.ConsumingPower = 0
        for compo in self.compos.values():
            #print(compo.name,compo.PowerConsumption,compo.Active)
            #衛星全体の電力状態を更新
            if compo.Active:
                self.RemainingPower = self.RemainingPower - compo.PowerConsumption
                self.ConsumingPower = self.ConsumingPower + compo.PowerConsumption
            else:
                self.RemainingPower = self.RemainingPower
                self.ConsumingPower = self.ConsumingPower
            #print(compo,self.compos[compo].state)
        #print("remain",self.RemainingPower, "consumed", self.ConsumingPower)
            
    def classify_links(self):
        for compo in self.compos:
            for p_ID in self.all_links.keys():
                #deep copyしてるから別物
                if p_ID in self.compos[compo].COM_link:
                    self.COMpinks[p_ID] = copy.deepcopy(self.all_links[p_ID])
                if p_ID in self.compos[compo].TEL_link:
                    self.TELlinks[p_ID] = copy.deepcopy(self.all_links[p_ID])
    
    #targetとなるTEL ID, COM ID(これらはリスト)受け取って，そこから考える
    #これもほんまはコマンドとテレメトリの組み合わせのリストとして受け取りたい
    def find_target_path(self, targetTEL_ID, targetCOM_ID):
        self.targetTEL_ID = targetTEL_ID
        self.targetCOM_ID = targetCOM_ID
        #pathIDとして定義されていないものが入った時の対応がない
        #検証対象のパスをリストで取る．これだけだと複数あって中にかぶりが合った時に困る
        targetTEL_paths = [j for j in [self.TEL[i].path for i in targetTEL_ID]]
        targetCOM_paths = [j for j in [self.COM[i].path for i in targetCOM_ID]]
        
        #かぶりの解消と，階層構造をなくす操作が必要
        one_layer_targetTELpath = []
        for path in targetTEL_paths:
            self.down_demension(path, one_layer_targetTELpath)
        #被りなくして追加
        self.targetTELpath.extend(list(dict.fromkeys(one_layer_targetTELpath)))
        #COM
        #junctionを取得
        #listの同一にあるのがペアだと仮定している．入力の仕方としてどのやり方がきれいなのか考える．
        #どっちも影響するときはどうするん？やっぱりimpact TEL IDから見なあかんかもな
        junction = []
        for i in range(len(targetTEL_ID)):
            junction.extend(self.find_junction(targetTEL_ID[i], targetCOM_ID[i]))
            #print(junction[i].name)
            
        targetCOM_route = []
        for COM_ID in targetCOM_ID:
            one_layer_targetCOMpath = []
            targetCOM_path = self.COM[COM_ID].path
            self.down_demension(targetCOM_path, one_layer_targetCOMpath)
            for compo in junction:
                self.trace_with_compo(compo, one_layer_targetCOMpath, targetCOM_route)
                #print(targetCOM_route)
        self.targetCOMpath.extend(reversed(list(dict.fromkeys(targetCOM_route))))
        
        print("targetTEL:",self.targetTELpath)
        print("targetCOM:",self.targetCOMpath)
        
    #テレメトリから不具合のトリガーとなったコマンドを探す場合
    #def find_trigerCOM
    
    #この時に確率の高いものだけを抽出し，抽出したものは消すようにすれば段階的探査ができる
    #こいつを使っているのはコマンド（テレメも）のパスの階層をなくすため
    def down_demension(self, path, one_layer_path):
        for link in path:
            #階層構造を馴らす,csvでくること前提になっている？
            self.recurrent_search(link,one_layer_path)
        #順番はどうするべきか考える．GSからの距離に対応してない
       # return one_layer_path
    
    def recurrent_search(self, link, one_layer_path):
        if (type(link) != list):
            one_layer_path.append(link)
            return 1
        #こっちに来たものはネストされていたもの
        #空判定
        elif (link):
            #popしてしまうと元が変更されてしまうのでdeepcopy
            link_for_pop = copy.deepcopy(link)
            p = link_for_pop.pop(0)
            one_layer_path.append(p)
            return self.recurrent_search(link_for_pop,one_layer_path)
        else:
            return 1
        
    #実際はここで与える順番として，持っているパスが短いものから行きたい．それが地上局に近いはずなので
    def search_TEL(self, TEL_ID):
        verify_candidates = {}
        #print("TELtarget:",self.targetTELpath)
        #各テレメトリのverify_linkを更新
        self.TEL[TEL_ID].find_check_TEL(self.targetTELpath)
        #受け渡す
        local_verify_candidate = {"TEL":self.TEL[TEL_ID].verify_linkID}
        self.get_propose_links((TEL_ID,), local_verify_candidate, verify_candidates)
        #リセット
        self.TEL[TEL_ID].verify_linkID = []
        return verify_candidates
                        
            
    #こいつ自体をSystemで呼ぶのが普通かもしれない
    def find_check_COM(self, COM_ID):
        #初期化
        self.COM[COM_ID].verify_flag = 0
        self.COM[COM_ID].candidateTELnum = 0
        if not self.targetCOMpath:
            return 0
        one_layer_COMpinks = []
        verify_candidates = {}
        #各コマンドの経路のみを対象にしている
        self.down_demension(self.COM[COM_ID].path, one_layer_COMpinks)
        #print(self.COM[COM_ID].name, "'s one layer'", one_layer_COMpinks)
        for TEL_ID in self.COM[COM_ID].impact_telID:
            #不具合検知のきっかけとなった組み合わせはあてにならないので，飛ばす
            if COM_ID in self.targetCOM_ID and TEL_ID in self.targetTEL_ID or not TEL_ID:
                continue
            #以下の実装はACTIONコマンドにしか適用できない
            #コマンドの経路の定義時にtargetのコンポまでの経路のみを作っておく．
            #そのコマンドがtarfetの状態を変化させるかどうかによって調べるか調べないかを見るようにする，
            junction = self.find_junction(TEL_ID, COM_ID)
           
            #プラスアルファで経路内のコンポに入力がある場合はそれも考える．
            candidate_COMpinks = []
            for end_compo in junction: #一応junctionが複数あるかもしれないという仮定
                #そのコマンドが状態を変化させるのかを確認
                #電源操作だけのコマンドにはFunctionがない
                #initial_Commandのものは状態がすでに変化しているとして見るので，飛ばさないようにする
                if COM_ID not in self.initial_COM:
                    if (not self.COM[COM_ID].target[0]["Function"]) and (self.COM[COM_ID].Active == end_compo.Active):
                        break
                    #コマンドがアクセスするFunctionと一致するものを調べる．コマンドがもつFunctionは一つに絞ったほうがいいかもしれない?
                    elif  self.COM[COM_ID].target[0]["Function"]:
                        func = self.COM[COM_ID].target[0]["Function"][0]#取り合えず一つのFunctionの時のみ実装
                        if func in end_compo.Function.keys():
                            if self.COM[COM_ID].Active == end_compo.Function[func]["Active"]:
                                break
                #GSに達したら出る
                #roup見つかれば1，なければ0が返る
                if self.trace_with_compo(end_compo, one_layer_COMpinks, candidate_COMpinks):
                    #ループがあって組み合わせが初めて検証できる
                    #ここでいいのか？
                    self.TEL[TEL_ID].find_check_TEL(self.targetTELpath)
                    break
                else:
                    continue
            #状態を変化させないコマンドのcandidateは空
            if candidate_COMpinks:
                #影響テレメトリとの経路に対応するパスから検証ポートを探す
                for p_ID in candidate_COMpinks:
                    if p_ID in self.targetCOMpath:
                        self.COM[COM_ID].verify_linkID.append(p_ID)
            
            #ココらへんで情報として保持する必要がある．これはSystem変数でもつか
            local_verify_candidate = {"COM": self.COM[COM_ID].verify_linkID, "TEL":self.TEL[TEL_ID].verify_linkID}
            self.get_propose_links((COM_ID,TEL_ID), local_verify_candidate, verify_candidates)
            #verify linkがないやつは表示しない
            if not self.COM[COM_ID].verify_linkID and not self.TEL[TEL_ID].verify_linkID:
                #print(self.COM[COM_ID].name, "&", self.TEL[TEL_ID].name, "empty")
                continue
            elif not self.TEL[TEL_ID].checked_flag:
                self.COM[COM_ID].verify_flag = 1
                #引き出しの多さ
                self.COM[COM_ID].candidateTELnum = self.COM[COM_ID].candidateTELnum + 1
            #今回の組み合わせによるものであることを示すため，一回リセットする
            self.COM[COM_ID].verify_linkID = []
            self.TEL[TEL_ID].verify_linkID = []
        #print(verify_candidates)
        return verify_candidates
    
    #candidatesに追加していく．段階ごとのものを全て持つようにする？
    #選択肢表示の
    def get_propose_links(self, propose_action, verify_candidate_links, candidates):
        #今の所したの条件分岐は意味ないが，入れ方を今後返るかもしれないので．
        #TELのみのとき
        if(len(propose_action)<2):
            candidates[propose_action] = verify_candidate_links
        #COMとTELの組み合わせ
        else:
            candidates[propose_action] = verify_candidate_links
        
    def find_junction(self, TEL_ID, COM_ID):
        junction = []
        ini_link = self.TEL[TEL_ID].path[0]
        for compo in self.compos.values():
            if ini_link in compo.TEL_link:
                #これだけでは本当にただのjunction．見つけたあとの判定でしっかりループになる経路のjunctionなのか判別しないといけない
                junction.append(compo) # listの方がいい？
        return junction
    
    #モデル生成の過程に関してもここを流用したいが難しそう．．
    def trace_with_compo(self, compo, one_layer_COMpinks, candidate_COMpinks):
        trace_candidate_COMpinks = copy.deepcopy(one_layer_COMpinks)
        if compo.name == "GS": #根元
            return 1 #何を返す？
        #コマンドの経路が分岐して合流することはないと考える．->あるよ．電源のとこ
        else:
            #まずこのコンポの前のパスを探す．コンポがもつポートは始点コンポが持つので．注意
            #このやり方ではダメ．複数パスの時に
            next_compo_name = " "
            #ここの探し方も辞書型なら直近のみを見れるのでは？
            #対象コマンドが持つコマンドポートを均したものを対象にしている
            for COMpink_ID in trace_candidate_COMpinks:
                COMpink = self.COMpinks[COMpink_ID]
                #現コンポを持つポートを探す
                if compo.name in COMpink.component:
                    next_compo_name = COMpink.component[0] \
                    if compo.name == COMpink.component[1] else COMpink.component[1]
                    next_compo = self.compos[next_compo_name]
                    flag = 0
                    #print("Temp next compo:",next_compo_name)
                    #ここまででnext compo仮ぎめ
                    for nCOMpink_ID in next_compo.COM_link:
                        if self.COMpinks[nCOMpink_ID].ID in trace_candidate_COMpinks \
                        and compo.name in self.COMpinks[nCOMpink_ID].component:
                            trace_candidate_COMpinks.remove(nCOMpink_ID)
                            candidate_COMpinks.append(nCOMpink_ID)
                            flag = True
                        else:
                            continue
                    if flag:
                        break
                    else:
                        #next compoの再探索が必要
                        next_compo_name = " "
                        continue
            #nothingのときはループがない→情報が返ってこないので検証の候補にできない
            if next_compo_name == " ":
                #print("nothing")
                return 0
            else:
                #print("next:",next_compo_name)
                return self.trace_with_compo(next_compo, trace_candidate_COMpinks, candidate_COMpinks)
        
    
    #検証済みかの確認を行うためのメソッド
    def check_links(self, target_path, links):
        target_linksID = []
        for p_ID in target_path:
            if not links[p_ID].valid:
                target_linksID.append(p_ID)
        return target_linksID
            
        
#リンクへのアクセスはID
class Link:
    #named_tupleを受け取る
    def __init__(self, link):
        self.name = link.Link_name
        self.component = [link.Compo1,link.Compo2]
        self.ID = link.ID
        self.medium = link.Medium
        self.probability = link.Probability
        self.valid = 0 #検証済みかどうか
        self.verifyCOMnum = 0
        #print(self.ID,self.component)

#各コンポごとにインスタンス化
#コンポへのアクセスは名前？
class Component:
    def __init__(self, compo): #,state
        com_linkID = compo.Com_linkID
        tel_linkID = compo.Tel_linkID
        #各ポートを全てリストとして持つ
        if (type(com_linkID) == str):
            #数値として扱う
            self.COM_link = [float(i) for i in com_linkID.split(',')]
        else:
            self.COM_link = [com_linkID]
        if (type(tel_linkID) == str):
            self.TEL_link = [float(i) for i in tel_linkID.split(',')]
        else:
            self.TEL_link = [tel_linkID]
        self.name = compo.Component
        self.state = {}
        #電源状態等をまとめて表現
        self.Active = True
        self.PowerConsumption = 0 #[W]
        self.Heat = 0
        self.Function = {}
        
    #stateの一部を変更した辞書を受取り，stateに関係する要素の更新を行う
    #直接更新した方がいい気がするが，一応名前はそのままにしておく．変えるかも
    def update_state(self,state_dict):
        self.state = state_dict
        #statusがないものは飛ばす
        if not len(self.state):
            return 0
        #電源状態
        self.Active = self.state["Active"]
        #電力消費量
        self.PowerConsumption = self.state["PowerConsumption"]["value"]
        self.Heat = self.state["Heat"] #+ or 0 or -?
        self.Function = self.state["Function"]
        return 1#この情報をどこかで使いたい
        
    #def find_link(self, link_list):
        
#固有なのはID
class Command(Satellite):
    def __init__(self, COM):
        self.ID = COM.ID
        #コマンドとテレメトリが持つmasterとなるpathは辞書が関係性を明確にできるのでいいかもしれない
        self.path = [COM.path]
        #pathの1列目以降は"_number"になっている．getattr()でアクセス
        for i in range(len(COM)-6):
            COM_attr = "_" + str(i+6)
            #print(COM_attr)
            compo = [float(i) for i in getattr(COM,COM_attr).split(',')] \
            if type(getattr(COM,COM_attr)) == str else getattr(COM,COM_attr)
            compo = 0 if compo == 0 else self.path.append(compo)
        self.name = COM.CommandName
        if type(COM.impact_Tel_ID)==str:
            self.impact_telID = [int(i) for i in COM.impact_Tel_ID.split(',')]
        else:
            self.impact_telID = [COM.impact_Tel_ID]
        self.verify_linkID= []
        self.verify_flag = 1 #verifyに使われる可能性があるかの判定
        self.candidateTELnum = 0
        #self.verify_TELlinkID= []
        #print(self.ID,":",self.path) #OK
        #self.init()
    
    def init_COM_type(self,type_dict):
        self.type = type_dict["type"]
        #compo, function, valueとかまとめて持つ
        self.target = type_dict["target"]
        #print(self.target)
        self.Active = type_dict["Active"]
        #print(self.target)
        #ACTIONコマンド
        
        #SET
        
        #GET
        
        
    #どんな形式で保持するのがいいのか考える．今の実装を考えるとリストにしないとめんどくさいが．
    def init(self, path_dict):
        #コマンドIDに応じて格納する処理？というか受け取るものを自分のIDのものだけにすればいい
        print(path_dict)
        
#固有なのはID
class Telemetry(Satellite):
    def __init__(self, TEL):
        self.ID = TEL.ID
        self.path = [TEL.path]
        #pathの1列目以降は"_number"になっている．getattr()でアクセス
        #offset = TEL.columns.get_loc('path')+2
        offset = 5
        for i in range(len(TEL)-offset):
            TEL_attr = "_" + str(i+offset)
            compo = [float(i) for i in getattr(TEL,TEL_attr).split(',')] \
            if type(getattr(TEL,TEL_attr)) == str else getattr(TEL,TEL_attr)
            compo = 0 if compo == 0 else self.path.append(compo)
        self.name = TEL.TelemetryName
        self.triger = TEL.TransitionTriger
        self.verify_linkID= []
        self.checked_flag = 0
    
    #targetTElpathが空なら終わる
    def find_check_TEL(self, targetTELpath):
        if not targetTELpath:
            return 0
        candidate_links = []
        #なんか必要やったが，名前以外の機能がないか確認
        super().down_demension(self.path, candidate_links)
        #print(self.name,"'s path:",candidate_links)
        #print("TELtarget:",targetTELpath)
        for p_ID in candidate_links:
            if p_ID in targetTELpath:
                self.verify_linkID.append(p_ID)
        #ここで更新されたパスを次に持ち込まないように情報を渡す必要がある
        #print(self.name, "'s verify links:", self.verify_linkID)
        

#これSystemクラスに埋め込んでもいいかも
def csv_read(file_path):
    df = pd.read_csv(file_path)
    return df.fillna(0) #NaNを0埋め
    
def json_read(file_path):
    json_open = open(file_path,'r')
    json_dict = json.load(json_open)
    return json_dict


In [72]:
compo_df = csv_read('csv/Components.csv')
link_df = csv_read('csv/Link.csv')
COM_df = csv_read('csv/Command.csv')
TEL_df = csv_read('csv/Telemetry.csv')
COM_path_dict = json_read('json/Command_path.json')
Compo_state_dict = json_read('json/Component_state.json')
COM_type_dict = json_read('json/Command_type.json')

def main():
    targetCOM = [14]
    targetTEL = [17]    #initial状態を利用する意味を成すのは状態量の導入をしてからな気がする．
    ini_COM = [14]
    #ini_TELも考える必要がありそう

    sat = Satellite(compo_df, link_df, COM_df, TEL_df, COM_type_dict)
    sat.init_state(ini_COM, Compo_state_dict)
    #実験
    #sat.COM[2].init(COM_path_dict['2'])
    sat.find_target_path(targetTEL, targetCOM)
    #print(sat.targetCOMpath)
    sys = System(sat)
    sys.verify_plan()
    #完了してたら原因不明
    print("faulty COMlink:", sys.sat.targetCOMpath, "faulty TELlink:",sys.sat.targetTELpath) #最終的な残り
    
main()

targetTEL: [40, 24, 8.0, 6.0, 2.0]
targetCOM: []
TELtarget: [40, 24, 8.0, 6.0, 2.0]
Telemetry 1 ( MOBC_Counter ) can verify following links
 [6, 2]

Please check MOBC_Counter
Input result(OK or NG)>>OK
TELtarget: [40, 24, 8.0]
Telemetry 2 ( TOBC_Counter ) can verify following links
 [8]

Please check TOBC_Counter
Input result(OK or NG)>>OK

 Check telemetries which influenced by initial Command state

COMtarget: [] TELtarget: [40, 24]


TypeError: 'int' object is not iterable